In [3]:
#High Variance = Large difference between test data errors. Often occurs with overfit ML models
#High Bias = Difference between train data error. Often occurs with underfit(simple, linear) ML models
#A model is said to be balanced if both the variance and bias is low
#Ensemble Learning: Dividing a dataset and running ML model on each and taking the average. Called bagging (RFC is a bagging technique). Especially useful if the dataset has high variance

In [46]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import svm, datasets
from sklearn.datasets import load_digits
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20,10) 

In [5]:
df = pd.read_csv('C:\\Users\\Guest_User\\Downloads\\diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [8]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [12]:
x = df.drop(['Outcome'], axis = 'columns')
y = df.Outcome

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xs = scaler.fit_transform(x)

In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(xs,y,test_size = 0.25, stratify = y, random_state = 10)

In [17]:
xtrain.shape

(576, 8)

In [18]:
xtest.shape

(192, 8)

In [30]:
dt = DecisionTreeClassifier()
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dt, x,y,cv = 5)
scores.mean()

0.7045581869111281

In [27]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 100,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0
)

In [28]:
bg.fit(xtrain,ytrain)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True, random_state=0)

In [29]:
bg.score(xtest,ytest)

0.7760416666666666

In [31]:
scores = cross_val_score(bg, x,y,cv = 5)
scores.mean()

0.7578728461081402

In [32]:
df = pd.read_csv('C:\\Users\\Guest_User\\Downloads\\heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [33]:
df1 = df[df.Cholesterol<=(df.Cholesterol.mean()+3*df.Cholesterol.std())]
df2 = df1[df1.Oldpeak<=(df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df3 = df2[df2.RestingBP<=(df2.RestingBP.mean()+3*df2.RestingBP.std())]
df4 = df3.copy()
df4.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df4.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df4.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)

df4.head()

C:\Users\Guest_User\AppData\Local\Temp\ipykernel_19920\2359806689.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df4.ExerciseAngina.replace(
C:\Users\Guest_User\AppData\Local\Temp\ipykernel_19920\2359806689.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4.ExerciseAngina.replace(
C:\Users

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,3,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,3,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,3,0


In [34]:
df5 = pd.get_dummies(df4, drop_first = True)
x = df5.drop(['HeartDisease'], axis = 'columns')
y = df5['HeartDisease']

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xs = scaler.fit_transform(x)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(xs,y,test_size = 0.25, random_state = 2) 

In [38]:
svc = svm.SVC()
svc.fit(xtrain,ytrain)
scores = cross_val_score(svc,x,y,cv = 5)
scores.mean()

0.6906445672191528

In [40]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
    estimator = svm.SVC(),
    n_estimators = 10,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0
)
scores = cross_val_score(bg,x,y,cv = 5)
scores.mean()

C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.6717249846531615

In [42]:
dt = DecisionTreeClassifier()
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dt,x,y,cv = 5)
scores.mean()

0.7171823204419889

In [43]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 10,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0
)
scores = cross_val_score(bg,x,y,cv = 5)
scores.mean()

C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Guest_User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.7804726826273788

In [44]:
rfc = RandomForestClassifier()

In [45]:
scores = cross_val_score(rfc,x,y,cv = 5)
scores.mean()

0.8181399631675875

In [ ]:
#Bagging does not impact SVM as SVM is a stable algo. Bagging works with high variance and unstable models (like decision trees)
#RFC uses bagging for both the data rows and columns